In [2]:
! pip -q install transformers
! pip install -q huggingface_hub
! pip -q install peft
! pip -q install accelerate
! pip -q install bitsandbytes
! pip -q install datasets
! pip install -q plotly

Initializing the huggingface 

In [4]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
tokenizer = AutoTokenizer.from_pretrained("NumbersStation/nsql-350M")
model = AutoModelForCausalLM.from_pretrained("NumbersStation/nsql-350M",load_in_8bit=True, device_map='auto')

tokenizer_config.json:   0%|          | 0.00/237 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/1.08k [00:00<?, ?B/s]

...

config.json:   0%|          | 0.00/1.01k [00:00<?, ?B/s]

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


pytorch_model.bin:   0%|          | 0.00/1.51G [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


generation_config.json:   0%|          | 0.00/115 [00:00<?, ?B/s]

Creating custom dataset from **NSText2SQL** dataset.
* * *

**Pipeline:**

dataset(huggingface object) --> pandas --> Filtering and Augmentation --> dataset(huggingface object)


In [5]:
import pandas as pd
from datasets import load_dataset

# load the dataset
dataset = load_dataset("NumbersStation/NSText2SQL",split="train")

# convert the dataset into a pandas dataframe
df = pd.DataFrame(dataset)

# print the shape and columns in the dataframe
print(f"Total Records in dataset : {df.shape[0]}")
print(f"Columns in dataset : {df.columns}")

Generating train split:   0%|          | 0/289288 [00:00<?, ? examples/s]

Total Records in dataset : 289288
Columns in dataset : Index(['instruction', 'output', 'source'], dtype='object')


**Analyzing the distrubution of instruction and output before filtering.**

In [6]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# create a dummy_df to store the length distrubution
dummy_df = pd.DataFrame(columns=['instruction_length', 'output_length'])

# Calculate the lengths of instruction and output columns
dummy_df['instruction_length'] = df['instruction'].apply(len)
dummy_df['output_length'] = df['output'].apply(len)

fig = make_subplots(rows=1, cols=2, subplot_titles=("Instruction Length", "Output Length"))

fig.add_trace(go.Histogram(
    x=dummy_df['instruction_length'],
    name='Instruction Length',
    opacity=0.75
), row=1, col=1)

fig.add_trace(go.Histogram(
    x=dummy_df['output_length'],
    name='Output Length',
    opacity=0.75
), row=1, col=2)


fig.update_layout(
    template='plotly_dark',
    title='Histogram of Instruction and Output Lengths',
    width=1000,
    height=600,
    showlegend=False
)

fig.update_xaxes(title_text="Length", row=1, col=1)
fig.update_yaxes(title_text="Count", row=1, col=1)
fig.update_xaxes(title_text="Length", row=1, col=2)
fig.update_yaxes(title_text="Count", row=1, col=2)

# Show the plot
fig.show()

**Apply the filtering**

In [7]:
import re
import random
def count_tables(instruction: str)-> int:
    """ Count the number of CREATE TABLE statements in the instruction """
    return len(re.findall(r'CREATE TABLE', instruction))


def replace_database(instruction:str)->str:
    """ Replace the valid SQLite database with a random database """
    databases = ['SQLite', 'MySQL', 'PostgreSQL']
    chosen_db = random.choice(databases)
    return re.sub(r'Using valid SQLite', f'Using valid {chosen_db}', instruction)

# filter the dataframe so that we have at least 3 tables and at most 10 tables.
filtered_df = df[(df['instruction'].apply(count_tables) > 3) & (df['instruction'].apply(count_tables) < 10)]

# add MySQL and PostgreSQL to the prompt.
filtered_df['instruction'] = filtered_df['instruction'].apply(replace_database)


# Calculate the number of rows removed
rows_removed = df.shape[0] - filtered_df.shape[0]

# Calculate the percentage of rows removed
percentage_removed = (rows_removed / df.shape[0]) * 100

# Print the results
print(f"Number of rows removed: {rows_removed}")
print(f"Percentage of rows removed: {percentage_removed:.2f}%")

del df
del dataset

Number of rows removed: 232083
Percentage of rows removed: 80.23%


/tmp/ipykernel_23/740343381.py:18: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



**After Filtering:**

In [8]:
# create a dummy_df to store the length distrubution
dummy_df = pd.DataFrame(columns=['instruction_length', 'output_length'])

# Calculate the lengths of instruction and output columns
dummy_df['instruction_length'] = filtered_df['instruction'].apply(len)
dummy_df['output_length'] = filtered_df['output'].apply(len)

fig = make_subplots(rows=1, cols=2, subplot_titles=("Instruction Length", "Output Length"))

fig.add_trace(go.Histogram(
    x=dummy_df['instruction_length'],
    name='Instruction Length',
    opacity=0.75
), row=1, col=1)

fig.add_trace(go.Histogram(
    x=dummy_df['output_length'],
    name='Output Length',
    opacity=0.75
), row=1, col=2)


fig.update_layout(
    template='plotly_dark',
    title='Histogram of Instruction and Output Lengths',
    width=1000,
    height=600,
    showlegend=False
)

fig.update_xaxes(title_text="Length", row=1, col=1)
fig.update_yaxes(title_text="Count", row=1, col=1)
fig.update_xaxes(title_text="Length", row=1, col=2)
fig.update_yaxes(title_text="Count", row=1, col=2)

# Show the plot
fig.show()

**Converting databack to dataset object**

In [9]:
from datasets import Dataset

filtered_dataset = Dataset.from_pandas(filtered_df)

filtered_dataset = filtered_dataset.remove_columns("__index_level_0__")


# do the train test split 
filtered_dataset = filtered_dataset.train_test_split(test_size=0.3)


**Pytorch Dataloader**

In [10]:
from datasets import load_dataset
from torch.utils.data import Dataset
import copy

class AIMISQLDataloder(Dataset):
    def __init__(self, mode='train', size=None, max_seq_length=2048):
        self.dataset = filtered_dataset

        if mode == 'train':
            self.dataset = self.dataset['train']
        elif mode == 'test':
            self.dataset = self.dataset['test']
        else:
            raise ValueError("Mode should be either 'train' or 'test'")
        
        if size:
            self.dataset = self.dataset.select(range(size))
        
        self.max_seq_length = max_seq_length
        self.tokenizer = tokenizer  

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, index):
        instruction = torch.tensor(self.tokenizer.encode(self.dataset[index]['instruction']), dtype=torch.int64)
        example = self.dataset[index]['instruction'] + self.dataset[index]["output"]
        example = self.tokenizer.encode(example)
        example.append(self.tokenizer.eos_token_id)
        padding = self.max_seq_length - len(example)
        example = torch.tensor(example, dtype=torch.int64)

        if padding < 0:
            example = example[:self.max_seq_length]
        else:
            example = torch.cat((example, torch.zeros(padding, dtype=torch.int64)))

        labels = copy.deepcopy(example)
        labels[: len(instruction)] = -100

        return {"input_ids": example, "labels": labels}

In [11]:
# get the dataset object
dataset = AIMISQLDataloder(size=30000, max_seq_length=1024,mode="train")
# dataset_test = AIMISQLDataloder(size=200, max_seq_length=1024,mode="test")


In [12]:
linear_layers = [
    "qkv_proj",
    "out_proj",
    "fc_in",
    "fc_out",
    "lm_head"
]

In [13]:
from peft import (
    get_peft_model,
    LoraConfig,
    TaskType,
    prepare_model_for_kbit_training,
)


model.train()

model = prepare_model_for_kbit_training(model)

lora_config = LoraConfig(
    task_type=TaskType.CAUSAL_LM,
    inference_mode=False,
    r=8,
    lora_alpha=32,
    lora_dropout=0.05,
    target_modules = linear_layers
)

model = get_peft_model(model, lora_config)

In [14]:
import numpy as np
from sklearn.metrics import accuracy_score, precision_score
from transformers import default_data_collator, Trainer, TrainingArguments


def compute_metrics(pred):
    logits, labels = pred
    predictions = np.argmax(logits, axis=-1)
    accuracy = accuracy_score(labels, predictions)
    precision = precision_score(labels, predictions, average='weighted')
    return {
        'accuracy': accuracy,
        'precision': precision
    }



output_dir = "training_run"

config = {
    'lora_config': lora_config,
    'learning_rate': 1e-4,
    'num_train_epochs': 2,
    'gradient_accumulation_steps': 2,
    'gradient_checkpointing': False,
}


training_args = TrainingArguments(
    output_dir=output_dir,
    overwrite_output_dir=True,
    logging_dir=f"{output_dir}/logs",
    logging_strategy="steps",
    logging_steps=5,
    optim="adamw_torch_fused",
    report_to="none",
    # evaluation_strategy = 'epoch',
    **{k:v for k,v in config.items() if k != 'lora_config'}
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset,
    # eval_dataset=dataset_test,
    data_collator=default_data_collator,
    # compute_metrics=compute_metrics
    
)


2024-07-26 17:53:47.811941: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-26 17:53:47.812063: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-26 17:53:47.944946: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [15]:
# Start training
trainer.train()

Token indices sequence length is longer than the specified maximum sequence length for this model (2142 > 2048). Running this sequence through the model will result in indexing errors
`use_cache=True` is incompatible with `config.gradient_checkpointing=True`. Setting `use_cache=False`...
/opt/conda/lib/python3.10/site-packages/torch/utils/checkpoint.py:429: UserWarning:

torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.

/opt/conda/lib/python3.10/site-packages/bitsandbytes/autograd/_functions.py:316: UserWarning:

MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization



Step,Training Loss
5,0.267100
10,0.064200
15,0.034400
20,0.010800
25,0.003700
30,0.001500
35,0.003200
40,0.001200
45,0.001200
50,0.000500


/opt/conda/lib/python3.10/site-packages/peft/utils/save_and_load.py:202: UserWarning:

Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.

/opt/conda/lib/python3.10/site-packages/torch/utils/checkpoint.py:429: UserWarning:

torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.

/opt/conda/lib/python3.10/site-packages/bitsandbytes/autograd/_functions.py:316: UserWarning:

MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization

/opt/conda/lib/python3.10/site-packages/peft/utils/save_and_load.py:202: UserWarning:

Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.

/opt/conda/lib/python3.10/

TrainOutput(global_step=3750, training_loss=0.0013150645711420413, metrics={'train_runtime': 31549.4975, 'train_samples_per_second': 1.902, 'train_steps_per_second': 0.119, 'total_flos': 1.132915064832e+17, 'train_loss': 0.0013150645711420413, 'epoch': 2.0})

In [16]:
from datetime import datetime
current_time = datetime.now().strftime("%Y-%m-%d_%H-%M")
model_name = f"model_{current_time}"


In [17]:
model.save_pretrained(model_name)
tokenizer.save_pretrained(model_name)

/opt/conda/lib/python3.10/site-packages/peft/utils/save_and_load.py:202: UserWarning:

Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.



('model_2024-07-27_02-39/tokenizer_config.json',
 'model_2024-07-27_02-39/special_tokens_map.json',
 'model_2024-07-27_02-39/vocab.json',
 'model_2024-07-27_02-39/merges.txt',
 'model_2024-07-27_02-39/added_tokens.json',
 'model_2024-07-27_02-39/tokenizer.json')

In [18]:
try:
    trainer.push_to_hub(model_name)
except:
  print("An exception occurred")

adapter_model.safetensors:   0%|          | 0.00/222M [00:00<?, ?B/s]

training_args.bin:   0%|          | 0.00/5.05k [00:00<?, ?B/s]

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]